In [1]:
import iris
import os
import shutil
import dask

In [2]:
SAMPLE_DATA_PATH="/data/cssp-china/sample-data-17-01-20/cssp_china_pp"

In [3]:
files = [os.path.join(SAMPLE_DATA_PATH, f) for f in os.listdir(SAMPLE_DATA_PATH)]

In [ ]:
# %%time
# iris.load(files[5],iris.AttributeConstraint(STASH='m01s16i203') )

In [ ]:
# c = [c for c in cubes if 'air_temperature' in c.name()][3]

In [ ]:
# c

In [ ]:
cubes = iris.load(files[1])


In [4]:
PARAM_NAME = 'air_temperature_mean_at_pressure_levels' # "precipitation_flux"
PARAM_CONSTRAINT = iris.AttributeConstraint(STASH='m01s16i203')  # "precipitation_flux"

In [5]:
from dask import bag as db

In [6]:
from collections import namedtuple

Paths = namedtuple('Paths',['old','new'])
def get_new_and_old_path(path, param_name):
    filename = os.path.basename(path)
    new_filename = filename[:-2] + 'nc' if filename[-2:].lower() == 'pp' else filename + '.nc'
    new_filename = f"{param_name}_{new_filename}"
    old_dir = os.path.dirname(path)
    new_dir = old_dir[:-2] + 'nc' if old_dir[:-2].lower() == 'pp' else old_dir + '_nc'
    new_path = os.path.join(new_dir, param_name, new_filename)
    return Paths(path, new_path)

eg_path = get_new_and_old_path(files[0],PARAM_NAME )
eg_path

Paths(old='/data/cssp-china/sample-data-17-01-20/cssp_china_pp/apepda.pa508i0.pp', new='/data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/air_temperature_mean_at_pressure_levels/air_temperature_mean_at_pressure_levels_apepda.pa508i0.nc')

In [7]:
def not_converted(paths):
    return not os.path.exists(paths.new)

not_converted(eg_path)

True

In [8]:
paths = eg_path

In [9]:
paths

Paths(old='/data/cssp-china/sample-data-17-01-20/cssp_china_pp/apepda.pa508i0.pp', new='/data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/air_temperature_mean_at_pressure_levels/air_temperature_mean_at_pressure_levels_apepda.pa508i0.nc')

In [ ]:
# import shutil
# import dask
# def local_convert(paths):
#     tmp_path = '/tmp/'+os.path.basename(paths.old)
#     tmp_path_new = '/tmp/'+os.path.basename(paths.new)
#     shutil.copy(paths.old, tmp_path)
    
#     with dask.config.set(scheduler='single-threaded'):
#         precipitation_flux
#         iris.fileformats.netcdf.save(cube, tmp_path_new, zlib=True, complevel=2)
    
    
#     shutil.copy(tmp_path_new, paths.new)
    
#     try:
#         os.remove(tmp_path_new)
#     except Exception:
#         print(f"could not delete {tmp_path_new}")
              
#     try:
#         os.remove(tmp_path)
#     except Exception:
#         print(f"could not delete {tmp_path}")

#     return paths

In [10]:
from pathlib import Path
def convert(paths, param_constraint):
    with dask.config.set(scheduler='single-threaded'):
        cubes = iris.load(paths.old, param_constraint)
        Path(os.path.dirname(paths.new)).mkdir(parents=True, exist_ok=True)
        iris.fileformats.netcdf.save(cubes, paths.new, zlib=True, complevel=2)
        return paths

In [ ]:
%%time
convert(eg_path, PARAM_CONSTRAINT)

In [ ]:
# %%time
# local_convert(eg_path)PARAM_CONSTRAINT

In [11]:
import distributed
import dask
from dask_kubernetes import KubeCluster
from dask import array as da

from dask.diagnostics import ProgressBar

from functools import partial



In [15]:
conversion = db.from_sequence(files).map(partial(get_new_and_old_path, param_name=PARAM_NAME)).filter(not_converted).map(partial(convert, param_constraint=PARAM_CONSTRAINT))
conversion

dask.bag<convert..., npartitions=116>

In [13]:
cluster = KubeCluster()
cluster.scale_up(100)
display(cluster)
# Connect dask to the cluster
client = distributed.Client(cluster)
client

distributed.dashboard.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://10.244.0.245:40829
distributed.scheduler - INFO -   dashboard at:                     :8787


distributed.scheduler - INFO - Receive client connection: Client-e7c03ee2-3eba-11ea-80f8-29016b7ac06d
distributed.core - INFO - Starting established connection


Client Scheduler: tcp://10.244.0.245:40829 Dashboard: /user/tam203/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


distributed.scheduler - INFO - Register tcp://10.244.5.21:39523
distributed.scheduler - INFO - Register tcp://10.244.5.20:43665
distributed.scheduler - INFO - Register tcp://10.244.5.17:42037
distributed.scheduler - INFO - Register tcp://10.244.5.16:35059
distributed.scheduler - INFO - Register tcp://10.244.5.22:34883
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.5.21:39523
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.5.20:43665
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.5.17:42037
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.5.16:35059
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.5.22:34883
distributed.c

In [ ]:
%%time
with ProgressBar():
    converted = conversion.compute()
converted

In [ ]:
!ls /data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/precipitation_flux/